
# Exploratory Data Analysis (EDA) - SQL

This notebook demonstrates how to perform data exploration using SQL queries on the SpaceX dataset.

## Objectives:
- Use SQL to find unique launch sites, successful missions, payload stats, etc.
- Analyze mission outcomes and booster performance.


## Import Required Libraries and Load Dataset

In [1]:
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

# Load data
df = pd.read_csv("spacex_clean_data.csv")
df.head()

,flight_no.,date_and_time_(utc),"version,_booster[h]",launch_site,payload[i],payload_mass,orbit,customer,launch_outcome,booster_landing
0,195,"January 3, 2023 14:56[17]",F9 B5 B1060‑15,"Cape Canaveral, SLC‑40",Transporter-6 (115 payload smallsat rideshare),Unknown[j],SSO,Various,Success,Success (LZ‑1)
1,196,"January 10, 2023 04:50[23]",F9 B5 B1076‑2,"Cape Canaveral, SLC‑40",OneWeb 16 (40 satellites),"6,000 kg (13,000 lb)",Polar LEO,OneWeb,Success,Success (LZ‑1)
2,FH 5,"January 15, 2023 22:56[29]",Falcon Heavy B5 B1070 (core),"Kennedy, LC‑39A",USSF-67 (CBAS-2 & LDPE-3A),"~3,750 kg (8,270 lb)",GEO,USSF,Success,No attempt
3,197,"January 18, 2023 12:24[33]",F9 B5 B1077‑2,"Cape Canaveral, SLC‑40",USA-343 (GPS-III SV06),"4,352 kg (9,595 lb)",MEO,USSF,Success,Success (JRTI)
4,198,"January 19, 2023 15:43[39]",F9 B5 B1075‑1,"Vandenberg, SLC‑4E",Starlink: Group 2-4 (51 satellites),"15,000 kg (33,000 lb)",LEO,SpaceX,Success,Success (OCISLY)


## Run SQL Queries

In [2]:
import re
#Clean payload_mass to extract numerical kg values
def extract_kg(payload):
    match = re.search(r"([\d,]+)\s*kg", str(payload))
    if match:
        return float(match.group(1).replace(",", ""))
    return None

df['payload_mass_clean'] = df['payload_mass'].apply(extract_kg)

# 1. Unique Launch Sites
q1 = pysqldf("SELECT DISTINCT launch_site FROM df")

# 2. Launch Sites beginning with 'CCA'
q2 = pysqldf("SELECT DISTINCT launch_site FROM df WHERE launch_site LIKE 'Cape Canaveral%' LIMIT 5")

# 3. Total payload from NASA
q3 = pysqldf("SELECT SUM(payload_mass_clean) AS total_payload FROM df WHERE customer LIKE '%NASA%'")

# 4. Average payload for F9 v1.1
q4 = pysqldf("SELECT AVG(payload_mass_clean) AS avg_payload FROM df WHERE `version,_booster[h]` LIKE '%F9%'")

# 5. First successful landing on ground pad
ground_pad_success_landings = ['Success (LZ‑1)', 'Success (LZ‑2)', 'Success (LZ‑4)']
q5 = df[df['booster_landing'].isin(ground_pad_success_landings)]['date_and_time_(utc)'].min()

# 6. Boosters that landed on drone ship with 4000 < payload < 6000
drone_ship_success_landings = ['Success (JRTI)', 'Success (OCISLY)', 'Success (ASOG)']
q6_fixed = df[(df['booster_landing'].isin(drone_ship_success_landings)) &
              (df['payload_mass_clean'] > 4000) &
              (df['payload_mass_clean'] < 6000)]['version,_booster[h]'].unique()
q6= pd.DataFrame(q6_fixed, columns=['version,_booster[h]'])

# 7. Total successful vs failure mission outcomes
q7 = pysqldf("SELECT launch_outcome, COUNT(*) AS count FROM df GROUP BY launch_outcome")

# 8. Booster with maximum payload
q8 = pysqldf("""
SELECT `version,_booster[h]`, MAX(payload_mass_clean) AS max_payload 
FROM df
""")

# 9. Failed landing outcomes on drone ship in 2015
q9 = pysqldf("""
SELECT `version,_booster[h]`, launch_site, `date_and_time_(utc)` 
FROM df 
WHERE booster_landing = 'Failure (drone ship)' 
AND `date_and_time_(utc)` LIKE '2015%'
""")

# 10. Ranked landing outcomes between 2010-06-04 and 2017-03-20
q10 = pysqldf("""
SELECT booster_landing, COUNT(*) AS count 
FROM df 
WHERE `date_and_time_(utc)` BETWEEN '2010-06-04' AND '2017-03-20' 
GROUP BY booster_landing 
ORDER BY count DESC
""")

## Display the output

In [3]:
print("---------------------------- Query 1: Unique Launch Sites ----------------------------")
print(q1)

print("\n---------------------------- Query 2: Launch Sites starting with 'CCA' ----------------------------")
print(q2)

print("\n---------------------------- Query 3: Total Payload from NASA ----------------------------")
print(q3)

---------------------------- Query 1: Unique Launch Sites ----------------------------
              launch_site
0  Cape Canaveral, SLC‑40
1         Kennedy, LC‑39A
2      Vandenberg, SLC‑4E
3      Vandenberg, SLC-4E
4    Kennedy, LC‑39A[493]

---------------------------- Query 2: Launch Sites starting with 'CCA' ----------------------------
              launch_site
0  Cape Canaveral, SLC‑40

---------------------------- Query 3: Total Payload from NASA ----------------------------
   total_payload
0       104758.0


In [4]:
print("\n---------------------------- Query 4: Average Payload for F9 v1.1 ----------------------------")
print(q4)

print("\n---------------------------- Query 5: First Successful Landing on Ground Pad ----------------------------")
print(q5)

print("\n---------------------------- Query 6: Boosters landed on Drone Ship with 4000 < Payload < 6000 ----------------------------")
print(q6)


---------------------------- Query 4: Average Payload for F9 v1.1 ----------------------------
    avg_payload
0  13702.638298

---------------------------- Query 5: First Successful Landing on Ground Pad ----------------------------
11 November 2024 17:22[411]

---------------------------- Query 6: Boosters landed on Drone Ship with 4000 < Payload < 6000 ----------------------------
    version,_booster[h]
0         F9 B5 B1077‑2
1         F9 B5 B1073‑6
2         F9 B5 B1077‑3
3         F9 B5 B1076‑4
4         F9 B5 B1078‑2
5        F9 B5 B1062‑14
6   F9 B5 B1067‑12[127]
7         F9 B5 B1077‑6
8         F9 B5 B1076‑9
9        F9 B5 B1076‑12
10        F9 B5 B1080‑9
11       F9 B5 B1076‑15
12       F9 B5 B1073‑19
13        F9 B5 B1085‑4
14       F9 B5 B1073‑20
15        F9 B5 B1092‑4


In [5]:

print("\n---------------------------- Query 7: Total Successful vs Failed Outcomes ----------------------------")
print(q7)

print("\n---------------------------- Query 8: Booster with Maximum Payload ----------------------------")
print(q8)

print("\n---------------------------- Query 9: Failed Drone Ship Landings in 2015 ----------------------------")
print(q9)



---------------------------- Query 7: Total Successful vs Failed Outcomes ----------------------------
  launch_outcome  count
0        Failure      1
1        Success    312

---------------------------- Query 8: Booster with Maximum Payload ----------------------------
  version,_booster[h]  max_payload
0      F9 B5 B1069‑13      17500.0

---------------------------- Query 9: Failed Drone Ship Landings in 2015 ----------------------------
Empty DataFrame
Columns: [version,_booster[h], launch_site, date_and_time_(utc)]
Index: []


In [6]:
print("\n---------------------------- Query 10: Ranked Landing Outcomes between 2010-06-04 and 2017-03-20 ----------------------------")
print(q10)


---------------------------- Query 10: Ranked Landing Outcomes between 2010-06-04 and 2017-03-20 ----------------------------
Empty DataFrame
Columns: [booster_landing, count]
Index: []
